<a href="https://colab.research.google.com/github/GeorgeShishkanov/university_project/blob/main/CV_project(first_part).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip "/content/CV_project1_data.zip"

Libraries

In [2]:
import os
import torch
import tarfile
import torchvision
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets.utils import download_url

Image folder to Dataset

In [4]:
transform_train = transforms.Compose([
    
    transforms.Resize((150,150)), #потому что vgg принимает 150*150
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))
    
])

#Augmentation.
transform_val = transforms.Compose([
    
    transforms.Resize((150,150)), #becasue vgg takes 150*150
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))
    
])

In [7]:
train_ds = ImageFolder('/content/data/train', transform=transform_train)
val_ds = ImageFolder('/content/data/valid', transform=transform_val)

In [10]:
image,label = train_ds[0]
print(image.shape,label)

torch.Size([3, 150, 150]) 0


Create a dataloader for all datasets

In [12]:
batch_size=128
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size, num_workers=2, pin_memory=True)

 Generate Class names

In [13]:
import pathlib

In [14]:
root = pathlib.Path("/content/data/train")
classes = sorted([j.name.split('/')[-1]for j in root.iterdir()])
classes

['bill_gates', 'elon_musk', 'jeff_bezos', 'mark_zuckerberg', 'steve_jobs']

Create accuracy function

In [15]:
def accuracy(outputs,labels):
  _,preds = torch.max(outputs,dim=1)
  return torch.tensor(torch.sum(preds==labels).item()/len(preds))

Download Pretrained model

In [ ]:
!pip install torchvision

In [44]:
from torchvision import models

In [45]:
modelvgg = models.vgg19(pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Freeze

In [26]:
for p in modelvgg.parameters():
    p.requires_grad = False

Model

In [30]:
#from torch.nn.modules.dropout import Dropout
#from torch.nn.modules.activation import ReLU
modelvgg.classifier = nn.Sequential(
    nn.Linear(in_features=25088,out_features=2048),
    nn.ReLU(),
    nn.Linear(in_features=2048,out_features=512),
    nn.ReLU(),
    nn.Dropout(p=0.6),

    nn.Linear(in_features=512,out_features=5),
    nn.Softmax(dim=1)

)

Create base class

In [31]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

Inherit base class

In [76]:
class IntelCnnModel(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = modelvgg
        
    def forward(self, xb):
        return self.network(xb)


Create object of inherited class

In [77]:
model = IntelCnnModel()

Training 

In [73]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()   #eval() is called to tell model that now it is validation mode and so don't perform stuff like dropout,backpropagation etc..
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.Adam):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train() #eval() is called to tell model that now it is training mode and so  perform stuff like dropout,backpropagation etc..
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [78]:
num_epochs = 5
opt_func = torch.optim.Adam
lr = 0.0001
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

KeyboardInterrupt: ignored